# Создание единой таблицы

In [13]:
import pandas as pd
from IPython.display import Latex
file_count = 78

def file_path(num, dir="data"):
    return f"{dir}/file{num}.csv"

In [15]:
# В файла может быть разное число колонок
# Исправим
for i in range(1, file_count + 1):
    lines = []
    with open(file_path(i), 'r', encoding='utf-8') as raw:
        for line in raw:
            lines.append(line.strip())
    max_semicolon_cnt = 0
    for line in lines:
        max_semicolon_cnt = max(max_semicolon_cnt, line.count(";"))
    with open(file_path(i, "data_cols_norm"), 'w') as file:
        for line in lines:
            semicolon_cnt = line.count(";")
            file.write(line + (max_semicolon_cnt - semicolon_cnt) * ";")
            file.write("\n")

In [18]:
df_s = []
headers = []
for i in range(1, file_count + 1):
    headers.append(pd.read_csv(file_path(i, "data_cols_norm"), nrows=1, encoding='cp1251', header=None, delimiter=";"))
    df_s.append(pd.read_csv(file_path(i, "data_cols_norm"), encoding='cp1251', delimiter=";"))


headers_all = [] # (index_start, index end], headers

headers_all.append([0])

master_table_cols = []
for i in range(1, file_count):
    header_prev = headers[i - 1].map(lambda x: [x] if pd.notnull(x) else []).sum(1).tolist()[0]
    header_curr = headers[i].map(lambda x: [x] if pd.notnull(x) else []).sum(1).tolist()[0]
    if len(header_prev) > len(header_curr) or header_prev != header_curr[:len(header_prev)]:
        headers_all[-1].append(i)
        headers_all[-1].append(header_prev)
        for header in header_prev:
            if header not in master_table_cols:
                master_table_cols.append(header)
        headers_all.append([i])

header_last = headers[file_count - 1].map(lambda x: [x] if pd.notnull(x) else []).sum(1).tolist()[0]
headers_all[-1].append(file_count)
headers_all[-1].append(header_last)

headers_all
for col in master_table_cols:
    display(Latex(col))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [19]:
import numpy as np


master_table_dict = dict()

for col in master_table_cols:
    master_table_dict[col] = []

for header_interval in headers_all:
    for file_num in range(header_interval[0], header_interval[1]):
        df_s[file_num].columns = header_interval[2][:df_s[file_num].shape[1]] + [""] * max(df_s[file_num].shape[1] - len(header_interval[2]), 0)
        for col in master_table_cols:
            if col in df_s[file_num].columns:
                master_table_dict[col].extend(df_s[file_num][col].tolist())
            else:
                master_table_dict[col].extend([np.nan] * df_s[file_num].shape[0])


master_table = pd.DataFrame(master_table_dict)
master_table.to_csv("master_table.csv", index=True, sep=';')